![](images/stock_data_tool.png)

### 📈 Tool 2: Stock performance data

You'll create a tool to load stock performance data from a local CSV file, filter it depending on the user's query, then return the results.

This tool will use stock _ticker symbols_, which are unique company IDs, to return the relevant information for that company. Here are the companies and symbols that have stock data available in your environment:

| Company Name | Ticker Symbol |
|--------------|---------------|
| Apple        | AAPL          |
| Microsoft    | MSFT          |
| Amazon       | AMZN          |
| Meta         | META          |
| Netflix      | NFLX          |
| Tesla        | TSLA          |

Note that this data isn't up-to-date, but you could adapt this to pull real-time data from an API like Yahoo Finance, rather than the CSVs. Try this out after completing the course!

**Run the installation below to configure your environment for the exercise.**

In [5]:
!pip install --quiet langchain-core==0.3.59


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


**Complete the code below to define a tool called `stock_data_tool()` that retrieves the last number of days (`num_days`) of stock performance data from local CSV files of a given company, identified by their stock ticker symbol (`company_ticker`)**

In [6]:
import os
from typing import Annotated
import pandas as pd
from langchain_core.tools import tool

@tool
def stock_data_tool(
    company_ticker: Annotated[str, "The ticker symbol of the company to retrieve their stock performance data."], 
    num_days: Annotated[int, "The number of days of stock data required to respond to the user query."]
) -> str:
    """
    Use this to look-up stock performance data for companies to retrieve a table from a CSV. You may need to convert company names into ticker symbols to call this function, e.g, Apple Inc. -> AAPL, and you may need to convert weeks, months, and years, into days.
    """
    
    # Load the CSV for the company requested
    file_path = f"data/{company_ticker}.csv"

    if os.path.exists(file_path) is False:
        return f"Sorry, but data for company {company_ticker} is not available. Please try Apple, Amazon, Meta, Microsoft, Netflix, or Tesla."
    
    stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

    # Ensure the index is in date format
    stock_df.index = stock_df.index.date
    
    # Maximum num_days supported by the dataset
    max_num_days = (stock_df.index.max() - stock_df.index.min()).days
    
    if num_days > max_num_days:
        return "Sorry, but this time period exceeds the data available. Please reduce it to continue."
    
    # Get the most recent date in the DataFrame
    final_date = stock_df.index.max()

    # Filter the DataFrame to get the last num_days of stock data
    filtered_df = stock_df[stock_df.index > (final_date - pd.Timedelta(days=num_days))]

    return f"Successfully executed the stock performance data retrieval tool to retrieve the last *{num_days} days* of data for company **{company_ticker}**:\n\n{filtered_df.to_markdown()}"

Because there's two inputs to this tool, we'll need to pass a dictionary of inputs to `.invoke()` instead of a string.

**Invoke your `wikipedia_tool()` on the `company_name` variable.**

In [7]:
retrieved_data = stock_data_tool.invoke({"company_ticker": "META", "num_days": 4})
print(retrieved_data)

Successfully executed the stock performance data retrieval tool to retrieve the last *4 days* of data for company **META**:

|            | Close/Last   |   Volume | Open    | High    | Low       |
|:-----------|:-------------|---------:|:--------|:--------|:----------|
| 2025-05-28 | $643.58      |  9042874 | $642.60 | $650.88 | $642.5472 |
| 2025-05-27 | $642.32      |  9508367 | $635.41 | $643.08 | $632.75   |


Because the table is in Markdown format, we can render it using the `display()` and `Markdown()` functions from `IPython.display`. This makes it possible to automate generating nicely formatted reports!

In [8]:
from IPython.display import display, Markdown
display(Markdown(retrieved_data))

Successfully executed the stock performance data retrieval tool to retrieve the last *4 days* of data for company **META**:

|            | Close/Last   |   Volume | Open    | High    | Low       |
|:-----------|:-------------|---------:|:--------|:--------|:----------|
| 2025-05-28 | $643.58      |  9042874 | $642.60 | $650.88 | $642.5472 |
| 2025-05-27 | $642.32      |  9508367 | $635.41 | $643.08 | $632.75   |